## Design patterns



## Composition

### Introduction

- *Composition* is another approach, different from *inheritance*, to structure code.
- Object Oriented Programming can be achieved with composition.
- Both are way to use, re-use and structure objects.

### Example: Engine Vehicle with inheritance

```python
class EngineVehicle(object):
    def __init__(self, engine_power):
        self.engine_power = engine_power
        
        
class Car(EngineVehicle):
    def __init__(self, engine_power):
        EngineVehicle.__init__(self, engine_power)
```

### Example: Engine vehicle with composition

```python
class Engine(object):
    def __init__(self, engine_power):
        self.engine_power = engine_power
        
        
class Car(object):
    def __init__(self, engine_power):
        self.engine = Engine(engine_power)
```

### Composition vs inheritance: what's the difference ?

- Inheritance defines a **is a** relationship  (or "behaves like")
- Composition defines a **has a** relationship

`(a car) is a (engine vehicle)`

`(a car) has an (engine)`


### Composition vs inheritance: which should I choose ?

It depends on what you want to do !  
Usually you can settle this with the following tests:

**Structure**
  - Does this new class have the same interface as the other class ? 

**Behavior**
  - Does this new class need only a part of the behavior of the other class ? 
  
**Substitution**
  - Can the code use `SpecializedClass` instead of `BasicClass` without breaking down ? (the affirmative indicates inheritance)

See also: [Liskov principle](https://en.wikipedia.org/wiki/Liskov_substitution_principle).



### To sum up

Inheritance makes the child class expose *all* the interface of the mother class.
  - If this is what you actually intended, then inheritance is advised (ex. GUI)
  - If this is not what was primarily intended (ex. you only need a part of the features of the other class), composition should be preferred.
  
Composition often requires lots of wrapping/cabling which can be done:
* explicitely (see the previous example)
* implicitly using  `__getattr__` but it makes the code far less readable





## Composition
